In [1]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_csv('../Data/Main/test.tsv', delimiter='\t')

In [61]:
question = df['Complete Question'].to_list()
answer = df['Answer Key'].to_list()

In [18]:
import openai
openai.api_key = 'xxx'

In [45]:
import time
res = []
for i, q in enumerate(question):
  completion = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    temperature = 0.8,
    max_tokens = 2000,
    messages = [
      {"role": "system", "content": "You are a student and only need to answer without explanation"},
      {"role": "user", "content": q}
    ]
  )
  res.append(completion.choices[0].message.content)
  if i%10 == 0:
    time.sleep(10)


In [59]:
# after mannually process the results to align the answer with A, B, C, D
f = open('result.txt', 'r')
Lines = f.readlines()

pred = []
for l in Lines:
    ch = l.strip()[2]
    pred.append(ch)

In [60]:
counts = 0
total = 500
wrong = []
for i in range(len(pred)):
    if pred[i] == answers[i]:
        counts += 1
    else:
        wrong.append((i, pred[i], answers[i]))
accuracy = counts/total

In [61]:
accuracy

0.74

In [62]:
wrong

[(2, 'A', 'C'),
 (8, 'B', 'D'),
 (12, 'D', 'C'),
 (19, 'C', 'B'),
 (28, 'A', 'D'),
 (29, 'C', 'B'),
 (32, 'D', 'B'),
 (38, 'A', 'D'),
 (50, 'D', 'A'),
 (51, 'C', 'D'),
 (54, 'A', 'C'),
 (66, 'D', 'A'),
 (71, 'D', 'C'),
 (74, 'D', 'C'),
 (75, 'A', 'C'),
 (89, 'B', 'A'),
 (98, 'B', 'A'),
 (99, 'D', 'C'),
 (103, 'B', 'D'),
 (108, 'C', 'D'),
 (109, 'A', 'C'),
 (116, 'A', 'D'),
 (117, 'A', 'D'),
 (125, 'B', 'A'),
 (128, 'D', 'B'),
 (133, 'D', 'A'),
 (134, 'A', 'D'),
 (137, 'A', 'B'),
 (138, 'A', 'B'),
 (140, 'A', 'C'),
 (141, 'D', 'A'),
 (146, 'C', 'A'),
 (148, 'D', 'A'),
 (154, 'B', 'A'),
 (166, 'U', 'A'),
 (168, 'A', 'C'),
 (174, 'A', 'B'),
 (178, 'A', 'D'),
 (191, 'A', 'B'),
 (192, 'A', 'D'),
 (194, 'A', 'C'),
 (195, 'D', 'B'),
 (200, 'C', 'D'),
 (203, 'D', 'A'),
 (211, 'D', 'B'),
 (213, 'A', 'B'),
 (216, 'B', 'C'),
 (232, 'D', 'A'),
 (235, 'D', 'B'),
 (237, 'D', 'A'),
 (250, 'B', 'D'),
 (256, 'A', 'D'),
 (261, 'C', 'B'),
 (262, 'A', 'B'),
 (270, 'D', 'C'),
 (277, 'D', 'C'),
 (282, 'A', 

In [47]:
import json

with open('../Data/Additional/test_complete.jsonl', 'r') as json_file:
    json_list = list(json_file)

questions = []
answers = []

for json_str in json_list:
    result = json.loads(json_str)
    question = result['fact1'] + '.'
    question += ' ' + result['question']['stem']
    for choice in result['question']['choices']:
        question += " (" + choice['label'] + ")" + " " + choice['text']
    questions.append(question)
    answers.append(result['answerKey'])

In [48]:
import time
import openai
openai.api_key = 'xxx'

In [49]:
res = []

In [52]:
for i, q in enumerate(questions[150:]):
  completion = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    temperature = 0.8,
    max_tokens = 2000,
    messages = [
      {"role": "system", "content": "You are a student and only need to answer A, B, C or D without explanation"},
      {"role": "user", "content": q}
    ]
  )
  res.append(completion.choices[0].message.content)
  if i%10 == 9:
    time.sleep(5)

In [55]:
pred = []
for a in res:
    if a[0] not in ['A', 'B', 'C', 'D']:
        pred.append(a[1])
    else:
        pred.append(a[0])

In [56]:
counts = 0
total = 500
wrong = []
for i in range(len(pred)):
    if pred[i] == answers[i]:
        counts += 1
    else:
        wrong.append((i, pred[i], answers[i]))
accuracy = counts/total